# 反爬虫技术

## 一、通过User-Agent来控制访问

无论是浏览器还是爬虫程序，在向服务器发起网络请求的时候，都会发过去一个头文件：headers,例如知乎的requests headers:    
<pre/>
Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8
Accept-Encoding:gzip, deflate, sdch, br
Accept-Language:zh-CN,zh;q=0.8,en;q=0.6,zh-TW;q=0.4,da;q=0.2,la;q=0.2
Cache-Control:max-age=0
Connection:keep-alive
Cookie: **
Host:https://www.zhihu.com/
Referer:https://www.zhihu.com/
Upgrade-Insecure-Requests:1
User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36
</pre>

这里的大多数字段都是向服务器**表明身份**用的    
对于爬虫程序来说，最需要注意的字段就是： "**User-Agent:**"      
很多网站会建立User-Agent白名单，只有属于正常范围的User-Agent才能够正常访问

In [2]:
###例如直接访问知乎

import requests
from bs4 import BeautifulSoup
import random

def get_page(url):
    try:
        r = requests.get(url, timeout = 30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return('Wrong in get_page()')
    
print(get_page('https://www.zhihu.com/'))

<html>
<head><title>400 Bad Request</title></head>
<body bgcolor="white">
<center><h1>400 Bad Request</h1></center>
<hr><center>openresty</center>
</body>
</html>



可以看到，我们的请求被拒绝了，并且返回了一个400的错误代码    
> 200 成功，代表服务器正常相应    
> 400 错误请求，代表服务器无法解析该请求    
> 403 未找到，服务器找不到请求的网页    
> 404 未找到，代表页面未找到    
> 500 服务器内部错误，代表服务器内部发生错误    
> 更多状态码参考 [HTTP基本原理](https://cuiqingcai.com/5465.html)


这里是因为我们本身的requests库的headers是这样的： 
<pre/>
{'Date': 'Sun, 05 May 2019 10:44:19 GMT', 
'Content-Type': 'text/html', 
'Content-Length': '170', 
'Connection': 'keep-alive', 
'Set-Cookie': 'tgw_l7_route=80f350dcd7c650b07bd7b485fcab5bf7; Expires=Sun, 05-May-2019 10:59:14 GMT; Path=/, _xsrf=qsWD0qeepnCXUb7N7hNKnweLHiTYw9sD; path=/; domain=zhihu.com; expires=Thu, 21-Oct-21 10:44:19 GMT', 'Server': 'ZWS', 'Vary': 'Accept-Encoding'}
</pre>

这里没有User-Agent字段，自然不被知乎的服务器所接受

In [5]:
#查看requests的headers
def get_headers(url):
    r = requests.get(url, timeout = 30)
    return r.headers

get_headers('https://www.zhihu.com/')

{'Date': 'Sun, 05 May 2019 10:44:19 GMT', 'Content-Type': 'text/html', 'Content-Length': '170', 'Connection': 'keep-alive', 'Set-Cookie': 'tgw_l7_route=80f350dcd7c650b07bd7b485fcab5bf7; Expires=Sun, 05-May-2019 10:59:14 GMT; Path=/, _xsrf=qsWD0qeepnCXUb7N7hNKnweLHiTYw9sD; path=/; domain=zhihu.com; expires=Thu, 21-Oct-21 10:44:19 GMT', 'Server': 'ZWS', 'Vary': 'Accept-Encoding'}

**解决办法**：   
我们可以自己设置一下User-Agent，或者更好的是，我们从一系列的User-Agent里随机挑选出一个符合标准的使用，代码如下

In [11]:
import requests
from bs4 import BeautifulSoup
import random

In [26]:
def get_agent():
    '''
    模拟headers的User-Agent字段
    返回一个随机的User-Agent字典类型的键值对
    '''
    
    agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
              'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;',
              'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv,2.0.1) Gecko/20100101 Firefox/4.0.1',
              'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
              'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
              'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)']
    fakeheader = {}
    fakeheader['User-Agent'] = agents[random.randint(0,len(agents)-1)]
    return fakeheader

def get_page(url):
    try:
        r = requests.get(url, timeout = 30, headers = get_agent())
        r.raise_for_status
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return('Something Wrong in get_page()')    

In [28]:
print(get_page('https://www.zhihu.com/'))

<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1" />
    <meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg" />
    <link rel="shortcut icon" type="image/x-icon" href="https://static.zhihu.com/static/favicon.ico" />
    <style>
      body {
        margin: 0;
        font-family: 'Helvetica Neue', Helvetica, 'PingFang SC', 'Hiragino Sans GB', 'Microsoft YaHei', Arial, sans-serif;
        font-size: 15px;
        color: #404040;
      }

      .Header {
        overflow: hidden;
        background-color: #fff;
        box-shadow: 0 3px 4px rgba(0, 0, 0, 0.06);
      }

      .Header-inner {
        width: 1008px;
        margin: 0 auto;
        padding: 0 16px;
      }

      .Header-logo {
        width: 64px;
        height: 30px;
        margin: 15px 0;
      }

      .Header-logo img {
        display: block;
        width: 100%;
        height: 1

## 二、通过IP限制来反爬虫

如果一个固定的ip在短时间内，快速大量的访问网站，那自然就会引起注意，管理员可以通过一些手段把这个ip给封了，爬虫自然就做不了什么了

**解决办法：**   
比较成熟的方式是：IP代理池    
简单的说就是通过ip代理，从不同的ip进行访问，这样就不会被封掉ip了    
可是ip代理的获取，本身就是一个很麻烦的事情。

In [30]:
def get_proxy():
    '''
    简答模拟代理池
    返回一个字典类型的键值对，
    '''
    proxy = ["http://116.211.143.11:80",
             "http://183.1.86.235:8118",
             "http://183.32.88.244:808",
             "http://121.40.42.35:9999",
             "http://222.94.148.210:808"]
    fakepxs = {}
    fakepxs['http'] = proxy[random.randint(0, len(proxy)-1)]
    
    return fakepxs

## 三、通过JS脚本来防止爬虫

这个可以说是终极的办法了，因为，爬虫终归只是一段程序，他并不能像人一样去应对各种变化，如验证码，滑动解锁之类的。    
举个例子：我曾经想爬一个分享百度云电影的网站，但是在进入网站之前，他会有一个验证页面来验证你是不是机器。    
他是怎么验证的呢：   
> 他会通过js代码生成一大段随机的数字，然后要求浏览器通过js的运算得出这一串数字的和，再返回给服务器.      

可想而知的是，这么简单和最基础的一个验证步骤，我们从前写的代码是完成不了的。

**解决办法：PhantomJS**

PhantomJS是一个Python包，他可以在没有图形界面的情况下，完全模拟一个”浏览器“，对你没看错，就是浏览器，这样的话，js脚本验证什么的再也不是问题了。

## 四、通过robots.txt来限制爬虫

说起来，世界上做爬虫最大最好的就是Google了，搜索引擎本身就是一个超级大的爬虫，Google开发出来爬虫24h不间断的在网上爬取着新的信息，并返回给数据库，但是这些搜索引擎的爬虫都遵守着一个协议：robots.txt

> robots.txt（统一小写）是一种存放于网站根目录下的ASCII编码的文本文件，它通常告诉网络搜索引擎的漫游器（又称网络蜘蛛），此网站中的哪些内容是不应被搜索引擎的漫游器获取的，哪些是可以被漫游器获取的。因为一些系统中的URL是大小写敏感的，所以robots.txt的文件名应统一为小写。robots.txt应放置于网站的根目录下。如果想单独定义搜索引擎的漫游器访问子目录时的行为，那么可以将自定的设置合并到根目录下的robots.txt，或者使用robots元数据（Metadata，又称元数据）。
robots.txt协议并不是一个规范，而只是约定俗成的，所以并不能保证网站的隐私。注意robots.txt是用字符串比较来确定是否获取URL，所以目录末尾有与没有斜杠“/”表示的是不同的URL。robots.txt允许使用类似"Disallow: *.gif"这样的通配符[1][2]。

In [ ]:
# 下面是京东的'robots.txt'
# https://www.jd.com/robots.txt
User-agent: * 
Disallow: /?* 
Disallow: /pop/*.html 
Disallow: /pinpai/*.html?* 
User-agent: EtaoSpider 
Disallow: / 
User-agent: HuihuiSpider 
Disallow: / 
User-agent: GwdangSpider 
Disallow: / 
User-agent: WochachaSpider 
Disallow: /

* User-agent: \* ：指的是对于任何的网络爬虫来源（定义为user-agent），也就是说无论你是什么样的网络爬虫都应该遵守如下的协议。    

* Disallow: /?\* ：不允许所有爬虫访问以问号开头的路径。   

* Disallow: /pop/\*.html ：不允许所有爬虫访问pop目录下的所有HTML页面。

* Disallow: /pinpai/\*.html?\* ：符合这个通配符的内容也是不允许任何网络爬虫访问的


   可以看到，jd的robots协议里明确的指出四个"user-agent”是禁止访问的，
事实上，这四个user-agent也是四个臭名昭著的恶性爬虫!